In [ ]:
import cv2
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from numpy import loadtxt
from tensorflow import keras
from sklearn import preprocessing
from  matplotlib import pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import normalize
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, model_from_json, Model
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import ResNet50, InceptionV3, EfficientNetB1, EfficientNetB1,EfficientNetB2, VGG19, EfficientNetB5, ConvNeXtXLarge,ConvNeXtLarge
from tensorflow.keras.layers import Input, Conv2D,Concatenate, LeakyReLU, Reshape, BatchNormalization, Flatten, Dense, Dropout, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D, Input,  MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -u '/content/drive/MyDrive/INDECS.zip' -d '/content/Images'
base_dir = '/content/INDECS'

Archive:  /content/drive/MyDrive/INDECS.zip


In [ ]:
base_cloudy = "/content/Images/INDECS/Cloudy"
base_sunny = "/content/Images/INDECS/Sunny"
base_night = "/content/Images/INDECS/Night"

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (112, 112))
    # img = img / 255.0
    img = np.array(img)
    img = np.expand_dims(img, axis=0)

    return img

In [ ]:
# Função para determinar o rótulo da imagem
def get_image_label(filename, labels):
    for i, label in enumerate(labels):
        if label in filename:
            return i
    return len(labels)

In [ ]:
def load_data(directory, labels):
    images = []
    image_paths = []
    image_labels = []

    for filename in sorted(os.listdir(directory)):  # Sort the filenames
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_paths.append(os.path.join(directory, filename))
            label = get_image_label(filename, labels)
            image_labels.append(label)

    for image_path in image_paths:
        image = preprocess_image(image_path)
        images.append(image)

    images = np.vstack(images)
    test_labels = np.array(image_labels)

    # one hot encode target values
    image_labels = to_categorical(image_labels, num_classes=5)
    print(image_labels.shape)
    # print(labels[5])

    # convert from integers to floats
    images = images.astype('float32')
    print(images.shape)

    return images, image_labels, test_labels

In [ ]:
labels_1 = ['rPR', 'rEO', 'rKT', 'rCR', 'rBO']
labels_2 = ['rPA', 'rEO', 'rKT', 'rCR', 'rBO']

X_train, y_train, test_labels_train = load_data(base_night,labels_1)

X_test_cloudy, y_test_cloudy, test_labels_cloudy  = load_data(base_cloudy,labels_1)
X_test_sunny, y_test_sunny, test_labels_sunny = load_data(base_sunny,labels_1)

(1092, 5)
(1092, 112, 112, 3)
(1080, 5)
(1080, 112, 112, 3)
(1080, 5)
(1080, 112, 112, 3)


In [ ]:
def build_classifier(input_shape):
    base_model = tf.keras.applications.ConvNeXtXLarge(include_top=False, input_shape=input_shape, weights='imagenet')
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    x = base_model.output
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output_layer = Dense(5, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs = output_layer)

In [ ]:
classifier_input = (112, 112, 3)

In [ ]:
# Define the fused model with both inputs and output
fused_model = build_classifier(classifier_input)
opt = Adam(learning_rate=0.0001)
fused_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fused_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 classifier_input (InputLay  [(None, 112, 112, 3)]        0         []                            
 er)                                                                                              
                                                                                                  
 detector_input (InputLayer  [(None, 46)]                 0         []                            
 )                                                                                                
                                                                                                  
 model_2 (Functional)        (None, 3, 3, 128)            3487054   ['classifier_input[0][0]']    
                                                          08                                

In [ ]:
# Early stopping para evitar overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [ ]:
# Define a callback de ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    filepath='melhor_modelo_night.h5',  # O caminho onde o modelo será salvo
    save_best_only=True,  # Salva apenas o melhor modelo
    monitor='val_accuracy',  # Métrica para determinar o "melhor"
    mode='max',
    verbose=1
)

In [ ]:
# Classes
classes = np.unique(np.argmax(y_train, axis=1))
class_weights = compute_class_weight('balanced', classes=classes, y=np.argmax(y_train, axis=1))
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Pesos das classes:", class_weights_dict)

Pesos das classes: {0: 1.011111111111111, 1: 1.3, 2: 1.011111111111111, 3: 0.56875, 4: 2.022222222222222}


In [ ]:
# class_weights_dict = {0: 6.0, 1: 1.4, 2: 1.0, 3: 0.56875, 4: 2.2}
class_weights_dict = {0: 4.5, 1: 1.3, 2: 1.01111111111111, 3: 0.56875, 4: 2.2}

history = fused_model.fit(
    X_train, y_train,
    batch_size=16, epochs=30, verbose=1,
    validation_data=(X_test_cloudy, y_test_cloudy),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
69/69 [==============================] - ETA: 0s - loss: 1.3100 - accuracy: 0.5137
Epoch 1: val_accuracy improved from -inf to 0.68148, saving model to melhor_modelo_night.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


69/69 [==============================] - 86s 898ms/step - loss: 1.3100 - accuracy: 0.5137 - val_loss: 0.7674 - val_accuracy: 0.6815
Epoch 2/30
69/69 [==============================] - ETA: 0s - loss: 0.3921 - accuracy: 0.8828
Epoch 2: val_accuracy improved from 0.68148 to 0.79352, saving model to melhor_modelo_night.h5
69/69 [==============================] - 60s 877ms/step - loss: 0.3921 - accuracy: 0.8828 - val_loss: 0.5491 - val_accuracy: 0.7935
Epoch 3/30
69/69 [==============================] - ETA: 0s - loss: 0.1726 - accuracy: 0.9661
Epoch 3: val_accuracy improved from 0.79352 to 0.83519, saving model to melhor_modelo_night.h5
69/69 [==============================] - 71s 1s/step - loss: 0.1726 - accuracy: 0.9661 - val_loss: 0.4689 - val_accuracy: 0.8352
Epoch 4/30
69/69 [==============================] - ETA: 0s - loss: 0.0812 - accuracy: 0.9899

In [ ]:
# Após o treinamento inicial, descongelar as últimas camadas e continuar o fine-tuning
for layer in fused_model.layers[-20:]:
    if hasattr(layer, 'trainable'):
        layer.trainable = True

In [ ]:
# Re-compilar o modelo com uma taxa de aprendizado muito baixa para o fine-tuning
fused_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continuar o treinamento para o fine-tuning com taxa de aprendizado reduzida
history_fitining = fused_model.fit(
    X_train, y_train,
    batch_size=16, epochs=30, verbose=1,
    validation_data=(X_test_cloudy, y_test_cloudy),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
# Evaluate the fused model on the combined test data
results_cloudy = fused_model.evaluate(X_test_cloudy, y_test_cloudy)

In [ ]:
results_sunny = fused_model.evaluate(X_test_sunny, y_test_sunny)

In [ ]:
pC = fused_model.predict(X_test_cloudy)
predC = np.argmax(pC, axis = 1)
pS = fused_model.predict(X_test_sunny)
predS = np.argmax(pS, axis = 1)

In [ ]:
print(classification_report(predC,test_labels_cloudy))

In [ ]:
print(classification_report(predS,test_labels_sunny))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # Show all ticks and label them with the respective list entries
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(test_labels_cloudy, predC, classes=labels_1, normalize=False, title='Confusion Matrix cloudy')
plt.show()

In [ ]:
plot_confusion_matrix(test_labels_sunny, predS, classes=labels_1, normalize=False, title='Confusion Matrix sunny')
plt.show()

In [ ]:
plot_model(fused_model, to_file='Model.png', show_shapes=True, show_layer_names=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the accuracy scores for each training condition
scores = {
    "Trained on Cloudy": {"Tested on Sunny": 80.41, "Tested on Night": 81.78},
    "Trained on Night": {"Tested on Sunny": 68.17, "Tested on Cloudy": 76.18},
    "Trained on Sunny": {"Tested on Cloudy": 79.20, "Tested on Night": 82.35}
}

# Set up the figure and axes for plotting
fig, axes = plt.subplots(1, 3, figsize=(15, 6), sharey=True)
fig.suptitle('Model Performance by Training and Testing Conditions')

# Colors and labels for the plots
colors = ['skyblue', 'lightgreen', 'lightcoral']
labels = list(scores.keys())

# Loop through the scores to plot
for ax, (label, tests) in zip(axes, scores.items()):
    # Data for the bars
    test_labels = list(tests.keys())
    accuracies = list(tests.values())

    # Create the bar plot
    bars = ax.bar(test_labels, accuracies, color=colors[:len(tests)])

    # Add the data labels on top of the bars
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

    ax.set_title(label)
    ax.set_ylim(0, 100)
    ax.set_ylabel('Accuracy (%)')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()